In [1]:
import json
from collections import Counter
import re

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, confusion_matrix

from tqdm import tqdm

from seq_helpers import *

should_downsample = False
should_include_kcs = True

In [2]:
# Set device to MPS (if using MacOS 12.3+). If not available, use CPU.
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Set device to CUDA (if using an Nvidia GPU). If not available, use CPU.
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Grid Search
We perform a grid search over N (the sequence length in the input data) and n_layers (the number of layers in each model). We select only these two hyperparemeters because we want to approximate the effect of varying the input sequence length on model performance (data source variation) AND the effect of increasing model complexity via n_layers. We could have chosen a different complexity hyperparameter (like hidden_size), but opt to choose only one due to computational constraints (adding one extra hyperparameter to the search increases runtime significantly).

In [3]:
hparams = {
    'batch_size': 32,
    'lr': 0.001,
    'epochs': 100,
    'input_size': 2
}

In [9]:
best_metrics = {
    'RNN': {
        'val': {'acc': 0, 'bal_acc': 0, 'f1_0': 0, 'f1_1': 0, 'sup_0': 0, 'sup_1': 0, 'auc': 0, 'epoch': 0},
        'test': {'acc': 0, 'bal_acc': 0, 'f1_0': 0, 'f1_1': 0, 'sup_0': 0, 'sup_1': 0, 'auc': 0, 'epoch': 0}
        },
    'LSTM': {
        'val': {'acc': 0, 'bal_acc': 0, 'f1_0': 0, 'f1_1': 0, 'sup_0': 0, 'sup_1': 0, 'auc': 0, 'epoch': 0},
        'test': {'acc': 0, 'bal_acc': 0, 'f1_0': 0, 'f1_1': 0, 'sup_0': 0, 'sup_1': 0, 'auc': 0, 'epoch': 0}
        },
    'Transformer': {
        'val': {'acc': 0, 'bal_acc': 0, 'f1_0': 0, 'f1_1': 0, 'sup_0': 0, 'sup_1': 0, 'auc': 0, 'epoch': 0},
        'test': {'acc': 0, 'bal_acc': 0, 'f1_0': 0, 'f1_1': 0, 'sup_0': 0, 'sup_1': 0, 'auc': 0, 'epoch': 0}
        },
}

for N in [5, 10, 20]:
    for n_layers in [1, 2, 5]:
        for model_name in ['RNN', 'LSTM', 'Transformer']:
            print(f"Training {model_name} with N = {N} and n_layers = {n_layers}")

            train_data = load_and_process_data(f'data_outputs/train_fold_1_n_{N}.json', seq_len = N)
            val_data = load_and_process_data(f'data_outputs/val_fold_1_n_{N}.json', seq_len = N)
            test_data = load_and_process_data(f'data_outputs/test_n_{N}.json', seq_len = N)

            train_dataset = StudentSequenceDataset(train_data, embedding_dim=0)
            val_dataset = StudentSequenceDataset(val_data, embedding_dim=0)
            test_dataset = StudentSequenceDataset(test_data, embedding_dim=0)

            train_data_loader = DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True)
            val_data_loader = DataLoader(val_dataset, batch_size=hparams['batch_size'], shuffle=False)
            test_data_loader = DataLoader(test_dataset, batch_size=hparams['batch_size'], shuffle=False)

            if model_name == 'RNN':
                model = SimpleRNN(input_size=hparams['input_size'], num_layers=n_layers).to(device)
            elif model_name == 'LSTM':
                model = SimpleLSTM(input_size=hparams['input_size'], num_layers=n_layers).to(device)
            elif model_name == 'Transformer':
                model = SimpleTransformer(input_size=hparams['input_size'], num_layers=n_layers).to(device)
            evaluator = ModelEvaluator({f'{model_name}_{N}_{n_layers}_epoch': model}, train_data_loader, val_data_loader, test_data_loader, device, epochs=hparams['epochs'], lr=hparams['lr'])
            evaluator.train_and_evaluate()
            val_metrics, test_metrics = evaluator.compute_best_metrics()

            if val_metrics['bal_acc'] > best_metrics[model_name]['val']['bal_acc']:
                best_metrics[model_name]['val'] = val_metrics
                best_metrics[model_name]['test'] = test_metrics
                print(f"New best {model_name} model with N = {N} and n_layers = {n_layers}!")
                print(f"    Val: {val_metrics}")
                print(f"    Test: {test_metrics}")


Training RNN with N = 5 and n_layers = 1


100%|██████████| 100/100 [08:31<00:00,  5.12s/it]


Best Metrics:
    Validation Set: RNN_5_1_epoch_92: {'acc': 0.7305726872246696, 'bal_acc': 0.6918040060299004, 'f1_0': 0.5992136304062909, 'f1_1': 0.797080291970803, 'sup_0': 2141, 'sup_1': 3534, 'auc': 0.6918040060299004, 'epoch': 92}
    Test Set: RNN_5_1_epoch_91: {'acc': 0.7424920127795527, 'bal_acc': 0.702279039485805, 'f1_0': 0.6121270452358035, 'f1_1': 0.8072692491630798, 'sup_0': 1192, 'sup_1': 1938, 'auc': 0.702279039485805, 'epoch': 91}

New best RNN model with N = 5 and n_layers = 1!
    Val: {'acc': 0.7305726872246696, 'bal_acc': 0.6918040060299004, 'f1_0': 0.5992136304062909, 'f1_1': 0.797080291970803, 'sup_0': 2141, 'sup_1': 3534, 'auc': 0.6918040060299004, 'epoch': 92}
    Test: {'acc': 0.7424920127795527, 'bal_acc': 0.702279039485805, 'f1_0': 0.6121270452358035, 'f1_1': 0.8072692491630798, 'sup_0': 1192, 'sup_1': 1938, 'auc': 0.702279039485805, 'epoch': 91}
Training LSTM with N = 5 and n_layers = 1


100%|██████████| 100/100 [06:42<00:00,  4.02s/it]


Best Metrics:
    Validation Set: LSTM_5_1_epoch_66: {'acc': 0.733568281938326, 'bal_acc': 0.6920919937818963, 'f1_0': 0.5970149253731345, 'f1_1': 0.801000263227165, 'sup_0': 2141, 'sup_1': 3534, 'auc': 0.6920919937818965, 'epoch': 66}
    Test Set: LSTM_5_1_epoch_77: {'acc': 0.736741214057508, 'bal_acc': 0.7010258448133757, 'f1_0': 0.6145930776426567, 'f1_1': 0.8000970402717127, 'sup_0': 1192, 'sup_1': 1938, 'auc': 0.7010258448133757, 'epoch': 77}

New best LSTM model with N = 5 and n_layers = 1!
    Val: {'acc': 0.733568281938326, 'bal_acc': 0.6920919937818963, 'f1_0': 0.5970149253731345, 'f1_1': 0.801000263227165, 'sup_0': 2141, 'sup_1': 3534, 'auc': 0.6920919937818965, 'epoch': 66}
    Test: {'acc': 0.736741214057508, 'bal_acc': 0.7010258448133757, 'f1_0': 0.6145930776426567, 'f1_1': 0.8000970402717127, 'sup_0': 1192, 'sup_1': 1938, 'auc': 0.7010258448133757, 'epoch': 77}
Training Transformer with N = 5 and n_layers = 1


...

Best Metrics:
    Validation Set: RNN_20_5_epoch_64: {'acc': 0.6907589055240062, 'bal_acc': 0.6247679481553988, 'f1_0': 0.4666073018699911, 'f1_1': 0.782260996001454, 'sup_0': 559, 'sup_1': 1378, 'auc': 0.6247679481553988, 'epoch': 64}
    Test Set: RNN_20_5_epoch_75: {'acc': 0.7017045454545454, 'bal_acc': 0.6461993888464477, 'f1_0': 0.5007923930269413, 'f1_1': 0.787305874409183, 'sup_0': 308, 'sup_1': 748, 'auc': 0.6461993888464477, 'epoch': 75}

Training LSTM with N = 20 and n_layers = 5


100%|██████████| 100/100 [05:11<00:00,  3.11s/it]


Best Metrics:
    Validation Set: LSTM_20_5_epoch_49: {'acc': 0.7000516262261228, 'bal_acc': 0.5393306780976812, 'f1_0': 0.23451910408432147, 'f1_1': 0.8134831460674158, 'sup_0': 559, 'sup_1': 1378, 'auc': 0.5393306780976812, 'epoch': 49}
    Test Set: LSTM_20_5_epoch_35: {'acc': 0.6884469696969697, 'bal_acc': 0.5356187929717342, 'f1_0': 0.24018475750577367, 'f1_1': 0.8040500297796308, 'sup_0': 308, 'sup_1': 748, 'auc': 0.5356187929717341, 'epoch': 35}

Training Transformer with N = 20 and n_layers = 5


100%|██████████| 100/100 [11:51<00:00,  7.12s/it]

Best Metrics:
    Validation Set: Transformer_20_5_epoch_0: {'acc': 0.7114093959731543, 'bal_acc': 0.5, 'f1_0': 0.0, 'f1_1': 0.8313725490196078, 'sup_0': 559, 'sup_1': 1378, 'auc': 0.5, 'epoch': 0}
    Test Set: Transformer_20_5_epoch_0: {'acc': 0.7083333333333334, 'bal_acc': 0.5, 'f1_0': 0.0, 'f1_1': 0.8292682926829268, 'sup_0': 308, 'sup_1': 748, 'auc': 0.5, 'epoch': 0}



# Confidence Score Demo
We select one of our best models (RNN with N=5 and n_layers=5), and use it to both predict the correctness of a random student's answer as well as a confidence in this prediction (approximated by the raw sigmoid output).

In [4]:
hparams = {
    'batch_size': 32,
    'lr': 0.001,
    'epochs': 100,
    'input_size': 2
}

In [5]:
model_name = 'RNN'
N = 5
n_layers = 5

print(f"Training {model_name} with N = {N} and n_layers = {n_layers}")

train_data = load_and_process_data(f'data_outputs/train_fold_1_n_{N}.json', seq_len = N)
val_data = load_and_process_data(f'data_outputs/val_fold_1_n_{N}.json', seq_len = N)
test_data = load_and_process_data(f'data_outputs/test_n_{N}.json', seq_len = N)

train_dataset = StudentSequenceDataset(train_data, embedding_dim=0)
val_dataset = StudentSequenceDataset(val_data, embedding_dim=0)
test_dataset = StudentSequenceDataset(test_data, embedding_dim=0)

train_data_loader = DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=hparams['batch_size'], shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=hparams['batch_size'], shuffle=False)

model = SimpleRNN(input_size=hparams['input_size'], num_layers=n_layers).to(device)

evaluator = ModelEvaluator({f'{model_name}_{N}_{n_layers}_epoch': model}, train_data_loader, val_data_loader, test_data_loader, device, epochs=hparams['epochs'], lr=hparams['lr'])
evaluator.train_and_evaluate()
val_metrics, test_metrics = evaluator.compute_best_metrics(save_dict=False)

Training RNN with N = 5 and n_layers = 5


100%|██████████| 100/100 [21:51<00:00, 13.11s/it]

Best Metrics:
    Validation Set: RNN_5_5_epoch_69: {'acc': 0.7309251101321586, 'bal_acc': 0.6920869715081122, 'f1_0': 0.5995279307631786, 'f1_1': 0.7973994958206182, 'sup_0': 2141, 'sup_1': 3534, 'auc': 0.6920869715081122, 'epoch': 69}
    Test Set: RNN_5_5_epoch_82: {'acc': 0.7408945686900958, 'bal_acc': 0.7014734452594178, 'f1_0': 0.6117759693633317, 'f1_1': 0.805562215296092, 'sup_0': 1192, 'sup_1': 1938, 'auc': 0.7014734452594178, 'epoch': 82}



In [10]:
labels, preds, confs = evaluator.predict(f'{model_name}_{N}_{n_layers}_epoch', test_data_loader, device, with_confidence=True)

# Print any 10 predictions and their confidences
for i in range(55, 66):
    print(f"True Label: {labels[i]}")
    print(f"Prediction: {preds[i]}")
    print(f"Confidence: {confs[i] * 100}%")
    if labels[i] != preds[i]:
        if confs[i] > 0.9:
            print("Verdict: Overconfident\n")
        else:
            print("Verdict: Balanced\n")
    
    if labels[i] == preds[i]:
        if confs[i] < 0.1:
            print("Verdict: Underconfident\n")
        else:
            print("Verdict: Balanced\n")

True Label: 1.0
Prediction: 1.0
Confidence: 99.99145269393921%
Verdict: Balanced

True Label: 0.0
Prediction: 0.0
Confidence: 0.0017738397218636237%
Verdict: Underconfident

True Label: 1.0
Prediction: 1.0
Confidence: 99.18718338012695%
Verdict: Balanced

True Label: 1.0
Prediction: 1.0
Confidence: 99.9940276145935%
Verdict: Balanced

True Label: 1.0
Prediction: 1.0
Confidence: 99.95296001434326%
Verdict: Balanced

True Label: 1.0
Prediction: 1.0
Confidence: 99.92734789848328%
Verdict: Balanced

True Label: 0.0
Prediction: 0.0
Confidence: 3.0384546789719025e-05%
Verdict: Underconfident

True Label: 1.0
Prediction: 1.0
Confidence: 99.97549653053284%
Verdict: Balanced

True Label: 0.0
Prediction: 1.0
Confidence: 99.97177720069885%
Verdict: Overconfident

True Label: 1.0
Prediction: 1.0
Confidence: 99.90384578704834%
Verdict: Balanced

True Label: 1.0
Prediction: 0.0
Confidence: 0.8072282187640667%
Verdict: Balanced

